In [11]:
import numpy as np
from abc import ABC, abstractmethod

class NaiveBayesBase(ABC):
    """Abstract Base Class for Naïve Bayes Classifiers."""
    
    def __init__(self):
        self.classes = None
        self.priors = {}

    def fit(self, X, y):
        """Train the Naïve Bayes classifier."""
        self.classes = np.unique(y)  # Get unique class labels
        self.priors = {c: np.mean(y == c) for c in self.classes}  # Compute class priors
        
        # Compute likelihood parameters based on subclass implementation
        self._compute_likelihood(X, y)

    def predict(self, X):
        """Predict class labels for given test data X."""
        predictions = []
        for x in X:
            class_probs = {}
            for c in self.classes:
                prior_log = np.log(self.priors[c])
                likelihood_log = self._compute_log_likelihood(x, c)
                class_probs[c] = prior_log + likelihood_log
            predictions.append(max(class_probs, key=class_probs.get))
        return np.array(predictions)

    @abstractmethod
    def _compute_likelihood(self, X, y):
        """Compute likelihood parameters based on the Naïve Bayes variant."""
        pass

    @abstractmethod
    def _compute_log_likelihood(self, x, c):
        """Compute log likelihood of a sample given class c."""
        pass



In [12]:
class GaussianNaiveBayes(NaiveBayesBase):
    """Gaussian Naïve Bayes Classifier (For Continuous Data)."""
    
    def __init__(self):
        super().__init__()
        self.means = {}
        self.variances = {}

    def _compute_likelihood(self, X, y):
        """Compute mean and variance for Gaussian distribution."""
        for c in self.classes:
            X_c = X[y == c]
            self.means[c] = np.mean(X_c, axis=0)
            self.variances[c] = np.var(X_c, axis=0)

    def _gaussian_pdf(self, x, mean, var):
        """Compute Gaussian probability density function."""
        epsilon = 1e-9  # Avoid division by zero
        coeff = 1 / np.sqrt(2 * np.pi * var + epsilon)
        exponent = np.exp(-((x - mean) ** 2) / (2 * var + epsilon))
        return coeff * exponent

    def _compute_log_likelihood(self, x, c):
        """Compute log likelihood using Gaussian distribution."""
        mean = self.means[c]
        var = self.variances[c]
        likelihoods = self._gaussian_pdf(x, mean, var)
        return np.sum(np.log(likelihoods + 1e-9))  # Sum of log likelihoods

In [13]:
class MultinomialNaiveBayes(NaiveBayesBase):
    """Multinomial Naïve Bayes Classifier (For Categorical/Text Data)."""
    
    def __init__(self):
        super().__init__()
        self.word_probs = {}

    def _compute_likelihood(self, X, y):
        """Compute likelihood probabilities for word occurrences using Laplace smoothing."""
        for c in self.classes:
            X_c = X[y == c]
            total_word_count = np.sum(X_c, axis=0) + 1  # Laplace smoothing (+1)
            self.word_probs[c] = total_word_count / np.sum(total_word_count)

    def _compute_log_likelihood(self, x, c):
        """Compute log likelihood using Multinomial distribution."""
        likelihoods = self.word_probs[c] ** x
        return np.sum(np.log(likelihoods + 1e-9))


In [14]:
class BernoulliNaiveBayes(NaiveBayesBase):
    """Bernoulli Naïve Bayes Classifier (For Binary Features)."""
    
    def __init__(self):
        super().__init__()
        self.probs = {}

    def _compute_likelihood(self, X, y):
        """Compute likelihood probabilities for binary features with Laplace smoothing."""
        for c in self.classes:
            X_c = X[y == c]
            self.probs[c] = (np.sum(X_c, axis=0) + 1) / (X_c.shape[0] + 2)  # Laplace smoothing

    def _compute_log_likelihood(self, x, c):
        """Compute log likelihood using Bernoulli distribution."""
        prob_c = self.probs[c]
        likelihoods = x * np.log(prob_c + 1e-9) + (1 - x) * np.log(1 - prob_c + 1e-9)
        return np.sum(likelihoods)